# Day 08. Exercise 00
# Binary classifier. Logistic regression

## 0. Imports

In [15]:
import pandas as pd

In [16]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import HoverTool
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## 1. Preprocessing

- Получите данные из файла [`checker-timestamp.csv`](https://drive.google.com/file/d/1pooM4cITtG249msX3GK_6W0eEiblzAf5/view?usp=sharing).


- Создайте фрейм данных `df` со столбцами: `date`, `am`, `pm`, `target`, где `date` - дата дня, `am` - количество коммитов за день до полудня (целое число), `pm` - количество коммитов за день после полудня (целое число), `target` - выходные/рабочий_день.

In [18]:
df = pd.read_csv('../data/checker_timestamp.csv', parse_dates=['timestamp'])
df['date'] = df['timestamp'].dt.date
df['hour'] = df['timestamp'].dt.hour
df['target'] = df['timestamp'].dt.dayofweek
df['am'] = df['hour'].apply(lambda x: 1 if x < 12 else 0)
df['pm'] = df['hour'].apply(lambda x: 1 if x >= 12 else 0)
df['target'] = df['target'].apply(lambda x: 'weekend' if x >= 5 else 'working_day')
df = df.groupby(['date', 'target']).agg({'am': 'sum', 'pm': 'sum'}).reset_index()

In [19]:
df.head()

,date,target,am,pm
0,2020-04-17,working_day,21,2
1,2020-04-18,weekend,1,68
2,2020-04-19,weekend,18,15
3,2020-04-20,working_day,2,23
4,2020-04-21,working_day,0,25


## 2. Exploratory analysis

- Постройте график, где `x` - утро, `y` - вечер, каждая точка - день, причем рабочие и выходные дни должны быть разного цвета.


- Как вы думаете, глядя на график, легко ли будет классифицировать дни, обладающие этими двумя признаками? Запишите свой ответ в ячейку markdown в конце этого раздела: «да, это легко» или „нет, это нелегко“.

In [20]:
def scatter_plot(x1, y1, x2, y2, title):
    fig = figure(title=title,
                 x_axis_label='count_am',
                 y_axis_label='count_pm',
                 width=600, height=400)

    fig.scatter(x1, y1, legend_label='working_day', color='blue', size=10, marker='circle')
    fig.scatter(x2, y2, legend_label='weekend', color='red', size=10, marker='circle')

    fig.legend.location = 'top_right'
    fig.legend.click_policy = 'hide'
    hover = HoverTool()
    hover.tooltips = [
        ("X", "@x"),
        ("Y", "@y"),
    ]
    fig.add_tools(hover)
    return fig

In [21]:
fig = scatter_plot(df[df['target'] == 'working_day']['am'],
            df[df['target'] == 'working_day']['pm'],
            df[df['target'] == 'weekend']['am'],
            df[df['target'] == 'weekend']['pm'],
            'am pm commit count')
show(fig)

нет, это не хорошо

## 3. Logistic regression

- Обучите логистическую регрессию на ваших данных, используя `am` и `pm`, параметры: `random state=21`, `fit_intercept=False`.


- Сделайте прогнозы для каждого дня вашего набора данных и добавьте их в dataframe с именем столбца `predict`.


- Сохраните кадр данных в файл в подпапке дня `data` с именем `am_pm.csv`.


- Нарисуйте еще один график, как вы делали это раньше, но цвет должен быть взят из `predict`.


- Как вы думаете, глядя на график, хорошо ли он предсказывает? Запишите свой ответ в ячейку markdown в конце этого раздела: «да, это хорошо»; «нет, это не хорошо».

In [22]:
model = LogisticRegression(random_state=21, fit_intercept=False)
model.fit(df[['am', 'pm']], df['target'])
df['predict'] = model.predict(df[['am', 'pm']])
df.to_csv('../data/am_pm.csv', index=False)

In [23]:
fig1 = scatter_plot(df[df['target'] == 'working_day']['am'],
            df[df['target'] == 'working_day']['pm'],
            df[df['target'] == 'weekend']['am'],
            df[df['target'] == 'weekend']['pm'], 'target')
fig2 = scatter_plot(df[df['predict'] == 'working_day']['am'],
            df[df['predict'] == 'working_day']['pm'],
            df[df['predict'] == 'weekend']['am'],
            df[df['predict'] == 'weekend']['pm'], 'target')
grid = gridplot([[fig1, fig2]])
show(grid)

нет, это не хорошо

## 4. Evaluation

- Рассчитайте `точность` для ваших предсказаний.


- Рассчитайте `точность` для наивного случая, когда каждое из ваших предсказаний - это значение самого популярного класса дня.


- Сравнивая точности, считаете ли вы, что классификатор сделал хорошие предсказания? Запишите свой ответ в ячейку markdown в конце раздела: «да, хорошо»; «нет, не хорошо».

In [24]:
print(f'accuracy = {accuracy_score(df['target'], df['predict'])}')

accuracy = 0.6285714285714286


In [25]:
print(f'accuracy = {accuracy_score([df['target'].mode()] * len(df['target']), df['predict'])}')

accuracy = 0.9142857142857143


нет, не хорошо